In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
import pickle

file_path = '/content/HR_Analytics.csv'
df = pd.read_csv(file_path)

df['Attrition'] = df['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0)

label_encoders = {}
categorical_columns = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime', 'SalarySlab']

for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

df = df.drop(columns=['EmpID', 'AgeGroup', 'EmployeeNumber', 'Over18'])

df.fillna(df.median(), inplace=True)

X = df.drop(columns=['Attrition'])
y = df['Attrition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

logreg = LogisticRegression()
logreg.fit(X_res, y_res)
logreg_pred = logreg.predict(X_test)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_res, y_res)
rf_pred = rf.predict(X_test)

gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_res, y_res)
gb_pred = gb.predict(X_test)

print("Logistic Regression Accuracy:", accuracy_score(y_test, logreg_pred))
print("Logistic Regression Confusion Matrix:\n", confusion_matrix(y_test, logreg_pred))
print("Logistic Regression Classification Report:\n", classification_report(y_test, logreg_pred))

print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_test, rf_pred))
print("Random Forest Classification Report:\n", classification_report(y_test, rf_pred))

print("Gradient Boosting Accuracy:", accuracy_score(y_test, gb_pred))
print("Gradient Boosting Confusion Matrix:\n", confusion_matrix(y_test, gb_pred))
print("Gradient Boosting Classification Report:\n", classification_report(y_test, gb_pred))

best_model = rf

with open('/content/attrition_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

print("Best model saved as attrition_model.pkl")


Logistic Regression Accuracy: 0.7364864864864865
Logistic Regression Confusion Matrix:
 [[183  66]
 [ 12  35]]
Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.73      0.82       249
           1       0.35      0.74      0.47        47

    accuracy                           0.74       296
   macro avg       0.64      0.74      0.65       296
weighted avg       0.84      0.74      0.77       296

Random Forest Accuracy: 0.847972972972973
Random Forest Confusion Matrix:
 [[237  12]
 [ 33  14]]
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.95      0.91       249
           1       0.54      0.30      0.38        47

    accuracy                           0.85       296
   macro avg       0.71      0.62      0.65       296
weighted avg       0.82      0.85      0.83       296

Gradient Boosting Accuracy: 0.8547297297297297
Gradient